In [1]:
import psycopg2
import subprocess
import os
from datetime import datetime
import threading
from concurrent.futures import ThreadPoolExecutor


############################
# NOTE: You must have a .pgpass entry to programmatically use pg_restore
# Run the following:
# > touch .pgpass
# > nano .pgpass
#       in the file, add your connection string:  "localhost:5432:mydatabase:myuser:mypassword"
# > chmod 600 ~/.pgpass
############################

In [2]:
############################
# Configure
############################

# postgres config
db_params = {
    "dbname": "postgres",
    "user": "postgres",
    "host": "localhost"
}

# export to subdirectory (in this working directory)
subdir = "Files"

# dump path
backup_file_path = "~/Downloads/usaspending-db_20230708"


In [3]:
# check if the subdirectory exists, if not, create it
if not os.path.exists(subdir):
    os.makedirs(subdir)


In [4]:
# this function takes a query that outputs a query and runs each sequentially
# its the dude playing a dude disguised as another dude
def nested_query_processor(query):
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            rows = cur.fetchall()
            
            # Iterate over the rows and execute each command
            for row in rows:
                alter_table_command = row[0]
                cur.execute(alter_table_command)

            conn.commit()


In [5]:
# get all tables in the dump (this is used to get chunks later)
cmd_list_tables = f"pg_restore --list {backup_file_path} | grep TABLE"
list_tables = subprocess.check_output(cmd_list_tables, shell=True).decode().splitlines()


In [6]:
# restore just the schema, including index, sequence & function definitions
cmd_restore_schema = f"pg_restore --no-owner --role=postgres --clean --schema-only -U {db_params['user']} -d {db_params['dbname']} {backup_file_path}"
subprocess.run(cmd_restore_schema, shell=True)

# this will output a bunch of warnings that look like errors
# if successful, the final output will be something like "CompletedProcess(...)"

pg_restore: while PROCESSING TOC:
pg_restore: from TOC entry 5509; 2606 17178 FK CONSTRAINT parent_award parent_award_parent_award_id_f3adaf05_fk_parent_award_award_id root
pg_restore: error: could not execute query: ERROR:  constraint "parent_award_parent_award_id_f3adaf05_fk_parent_award_award_id" of relation "parent_award" does not exist
Command was: ALTER TABLE ONLY rpt.parent_award DROP CONSTRAINT parent_award_parent_award_id_f3adaf05_fk_parent_award_award_id;
pg_restore: from TOC entry 5486; 2606 17248 FK CONSTRAINT treasury_appropriation_account treasury_appropriati_funding_toptier_agen_d9eb7c77_fk_toptier_a root
pg_restore: error: could not execute query: ERROR:  constraint "treasury_appropriati_funding_toptier_agen_d9eb7c77_fk_toptier_a" of relation "treasury_appropriation_account" does not exist
Command was: ALTER TABLE ONLY public.treasury_appropriation_account DROP CONSTRAINT treasury_appropriati_funding_toptier_agen_d9eb7c77_fk_toptier_a;
pg_restore: from TOC entry 5485; 2

CompletedProcess(args='pg_restore --no-owner --role=postgres --clean --schema-only -U postgres -d postgres ~/Downloads/usaspending-db_20230708', returncode=1)

In [7]:
# now we have the full scaffolding, remove any constraints

remove_constraints = """
    select 
        format('alter table %I.%I drop constraint if exists %I cascade;',table_schema, table_name, constraint_name)
    from information_schema.constraint_table_usage
    where table_schema not in ('pg_catalog')
    ;"""

nested_query_processor(remove_constraints)

In [8]:
# remove indexes with other index dependencies first
# todo: create a topical graph to handle nested dependencies

remove_d = """
    select 
        distinct 
        format('drop index if exists %I.%I cascade;', n1.nspname, c1.relname) as com
    from
        pg_catalog.pg_depend d
    join pg_catalog.pg_class c1 on c1.oid = d.refobjid
    join pg_catalog.pg_class c2 on c2.oid = d.objid
    join pg_catalog.pg_namespace n1 on c1.relnamespace = n1.oid
    join pg_catalog.pg_namespace n2 on c2.relnamespace = n2.oid
    join pg_catalog.pg_indexes as i on n1.nspname = i.schemaname and c1.relname = i.indexname
    where  c2.relkind = 'i' -- this will only include index objects
        and c1.relkind not in ('r')
        and n1.nspname not in ('pg_catalog','information_schema')
        and n1.nspname !~ '^pg_toast'::text
    ;"""

nested_query_processor(remove_d)

In [9]:
# remove all remaining indexes

remove_i = """
    select
        format('drop index if exists %I.%I cascade;', n.nspname, c_ind.relname)
    from pg_catalog.pg_index ind
    join pg_catalog.pg_class c_ind on c_ind.oid = ind.indexrelid
    join pg_catalog.pg_namespace n on n.oid = c_ind.relnamespace
    left join pg_catalog.pg_constraint cons on cons.conindid = ind.indexrelid
    where 1=1
        and n.nspname not in ('pg_catalog','information_schema')
        and n.nspname !~ '^pg_toast'::text
        and cons.oid is null
    ;"""

nested_query_processor(remove_i)

In [10]:
# main process to restore table, export to csv, gzip, delete csv, truncate table

def process_tables(list_tables):
    with psycopg2.connect(**db_params) as conn:
        with conn.cursor() as cur:
            for table_line in list_tables:
                parts = table_line.split()
                if 'TABLE' in parts[3] and 'DATA' in parts[4]:
                    table_schema = parts[5]
                    table_name = parts[6]
                    full_table_name = f"{table_schema}.{table_name}"

                    print(f"Restoring Table: {full_table_name} @ ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                    cmd_restore_table = f"pg_restore --no-owner --role=postgres -U {db_params['user']} -d {db_params['dbname']} -t {table_name} {backup_file_path}"
                    subprocess.run(cmd_restore_table, shell=True)

                    csv_file_path = os.path.join(subdir, f"{full_table_name.replace('.', '_')}.csv")
                    gz_file_path = os.path.join(subdir, f"{full_table_name.replace('.', '_')}.csv.gz")

                    #print(f"Exporting CSV: {full_table_name}")
                    with open(csv_file_path, 'w') as f:
                        cur.copy_expert(f"COPY {full_table_name} TO STDOUT WITH CSV HEADER", f)

                    #print(f"Gzip: {full_table_name}")
                    cmd_gzip_and_split = f"gzip -c {csv_file_path} | split -b 1000m - {gz_file_path}_part_"
                    subprocess.run(cmd_gzip_and_split, shell=True)

                    #print(f"Deleted original CSV: {csv_file_path}")
                    os.remove(csv_file_path)

                    cur.execute(f"truncate table {full_table_name}")
                    print(f"{full_table_name} Completed @ ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

                    conn.commit()

In [11]:
# create chunks of tables for threading
tables_at_a_time = 10 # set concurrency
chunks = [list_tables[i:i+tables_at_a_time] for i in range(0, len(list_tables), tables_at_a_time)]


In [ ]:
# execute main process
# thread pool to process chunks of tables concurrently
with ThreadPoolExecutor(max_workers=tables_at_a_time) as executor:
    executor.map(process_tables, chunks)

# you'll get a lot of pg_restore warnings; they can be ignored.

Restoring Table: int.duns @  2023-07-23 10:48:09
Restoring Table: public.django_content_type @  2023-07-23 10:48:09
Restoring Table: public.overall_totals @  2023-07-23 10:48:09
Restoring Table: public.auth_user_user_permissions @  2023-07-23 10:48:09
Restoring Table: rpt.transaction_search_fpds @  2023-07-23 10:48:09
Restoring Table: public.reporting_agency_overview @  2023-07-23 10:48:09
Restoring Table: public.uei_crosswalk_2021 @  2023-07-23 10:48:09
Restoring Table: public.financial_accounts_by_program_activity_object_class @  2023-07-23 10:48:09
public.auth_user_user_permissions Completed @  2023-07-23 10:48:09
Restoring Table: public.award_category @  2023-07-23 10:48:09
public.django_content_type Completed @  2023-07-23 10:48:09
Restoring Table: public.django_migrations @  2023-07-23 10:48:09
public.overall_totals Completed @  2023-07-23 10:48:09
Restoring Table: public.psc @  2023-07-23 10:48:09
public.reporting_agency_overview Completed @  2023-07-23 10:48:09
Restoring Table:

pg_restore: while PROCESSING TOC:
pg_restore: from TOC entry 551; 1259 16885 TABLE django_content_type root
pg_restore: error: could not execute query: ERROR:  relation "django_content_type" already exists
Command was: CREATE TABLE public.django_content_type (
    id integer NOT NULL,
    app_label character varying(100) NOT NULL,
    model character varying(100) NOT NULL
);


pg_restore: while PROCESSING TOC:
pg_restore: pg_restore:pg_restore: while PROCESSING TOC:
 pg_restore:while PROCESSING TOC:
pg_restore: from TOC entry 534; 1259 16534 TABLE overall_totals root
pg_restore:  from TOC entry 612; 1259 1583379388 TABLE reporting_agency_overview root
pg_restore: error: from TOC entry 590; 1259 186220831 TABLE duns root
pg_restore: error: error: could not execute query: ERROR:  relation "overall_totals" already exists
Command was: CREATE TABLE public.overall_totals (
    id integer NOT NULL,
    create_date timestamp with time zone,
    update_date timestamp with time zone,
    fiscal_

public.bureau_title_lookup Completed @  2023-07-23 10:48:09
Restoring Table: public.c_to_d_linkage_updates @  2023-07-23 10:48:09
public.c_to_d_linkage_updates Completed @  2023-07-23 10:48:10
Restoring Table: public.cgac @  2023-07-23 10:48:10
public.cgac Completed @  2023-07-23 10:48:10
Restoring Table: public.dabs_loader_queue @  2023-07-23 10:48:10
public.dabs_loader_queue Completed @  2023-07-23 10:48:10
Restoring Table: public.dabs_submission_window_schedule @  2023-07-23 10:48:10
public.dabs_submission_window_schedule Completed @  2023-07-23 10:48:10
Restoring Table: public.disaster_emergency_fund_code @  2023-07-23 10:48:10


pg_restore: while PROCESSING TOC:
pg_restore: from TOC entry 648; 1259 2732485752 TABLE dabs_submission_window_schedule root
pg_restore: error: could not execute query: ERROR:  relation "dabs_submission_window_schedule" already exists
Command was: CREATE TABLE public.dabs_submission_window_schedule (
    id integer NOT NULL,
    period_start_date timestamp with time zone NOT NULL,
    period_end_date timestamp with time zone NOT NULL,
    submission_start_date timestamp with time zone NOT NULL,
    submission_due_date timestamp with time zone NOT NULL,
    certification_due_date timestamp with time zone NOT NULL,
    submission_reveal_date timestamp with time zone NOT NULL,
    submission_fiscal_year integer NOT NULL,
    submission_fiscal_quarter integer NOT NULL,
    submission_fiscal_month integer NOT NULL,
    is_quarter boolean NOT NULL
);


pg_restore: warning: errors ignored on restore: 1
pg_restore: while PROCESSING TOC:
pg_restore: from TOC entry 645; 1259 2594475716 TABLE dis

public.disaster_emergency_fund_code Completed @  2023-07-23 10:48:10
Restoring Table: public.django_admin_log @  2023-07-23 10:48:10
public.django_admin_log Completed @  2023-07-23 10:48:10


...

public.historical_appropriation_account_balances Completed @  2023-07-23 10:51:44
Restoring Table: public.job_status @  2023-07-23 10:51:44
public.job_status Completed @  2023-07-23 10:51:45
Restoring Table: public.naics @  2023-07-23 10:51:45
public.naics Completed @  2023-07-23 10:51:45
Restoring Table: public.object_class @  2023-07-23 10:51:45


pg_restore: while PROCESSING TOC:
pg_restore: from TOC entry 577; 1259 17562 TABLE job_status root
pg_restore: error: could not execute query: ERROR:  relation "job_status" already exists
Command was: CREATE TABLE public.job_status (
    job_status_id integer NOT NULL,
    name text NOT NULL,
    description text,
    create_date timestamp with time zone,
    update_date timestamp with time zone
);


pg_restore: warning: errors ignored on restore: 1
pg_restore: while PROCESSING TOC:
pg_restore: from TOC entry 572; 1259 17528 TABLE naics root
pg_restore: error: could not execute query: ERROR:  relation "naics" already exists
Command was: CREATE TABLE public.naics (
    code text NOT NULL,
    description text NOT NULL,
    year integer NOT NULL
);


pg_restore: warning: errors ignored on restore: 1
pg_restore: while PROCESSING TOC:
pg_restore: from TOC entry 532; 1259 16512 TABLE object_class root
pg_restore: error: could not execute query: ERROR:  relation "object_class" already exists

public.object_class Completed @  2023-07-23 10:51:45
Restoring Table: public.office @  2023-07-23 10:51:45


pg_restore: warning: errors ignored on restore: 1


public.office Completed @  2023-07-23 10:51:46
